In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt

In [52]:
#load all aggregated csvs: UCDP, GDELT, Phoenix
path = '/Users/sabinejoseph/Downloads/kfe-sabinejo-patch-1/'

def csv_to_df(path, filename):
    df = pd.read_csv(path + filename, sep = ',', low_memory=False)
    df = df.reset_index(drop=True)
    return df

df_UCDP = csv_to_df(path, 'Agg_UCDP_daily.csv')
df_Phoe = csv_to_df(path, 'Phoenix_NaMo_Agg_subset.csv')
del df_Phoe['Unnamed: 0']
#df_GDELT = csv_to_df(path, 'GDELT_NaMo_subset.csv')

In [5]:
# read excel with country names and codes
# return list with country codes relevant for UCDP
country_code_pathfile = '/Users/sabinejoseph/Downloads/kfe-sabinejo-patch-1/Input/Country_codes_NAMO.xlsx'
country_code_column_name = 'UCDP_country_codes'
sheet = 'Sheet1'

def country_codes_from_excel(country_codes, sheet_num, column_name):
    countries = pd.ExcelFile(country_codes)
    countries = countries.parse(sheet_num)
    return list(countries[column_name]) # UCDP uses Gleditsch and Ward country codes

CCS = country_codes_from_excel(country_code_pathfile, sheet, country_code_column_name)
CC3 = country_codes_from_excel(country_code_pathfile, sheet, 'Country_3')
CC2 = country_codes_from_excel(country_code_pathfile, sheet, 'Country_2')

In [63]:
# create empty df
new_format = '%Y-%m-%d'
#old_format = '%Y-%m-%d %H:%M:%S'
start_date = '2014-01-01 00:00:00'
end_date = '2017-08-01 00:00:00'
CC_col_names = ['CC3', 'CCS', 'CC2']
CC_lists = [CC3, CCS, CC2]

def create_empty_df(date_format, start_date, end_date, CC_col_names, CC_lists):
    all_days = pd.date_range(start_date, end_date, freq='D')

    df = pd.DataFrame(index=pd.date_range(start = start_date, end = end_date, freq='90Min'))
    df.index = [df.index[i].strftime(new_format) for i in range(0, len(df.index)) if i is not None]
    df = df[:-1]
    
    for i in range(0, len(CC_col_names)):
        temp = CC_lists[i] * (len(df.index)/len(CC3))
        df[CC_col_names[i]] = temp
        df.is_copy = False
        
    return df

df = create_empty_df(new_format, start_date, end_date, CC_col_names, CC_lists)
df = df.reset_index()
df = df.rename(columns={'index': 'date_start'})

In [138]:
# create empty columns to extend combined df based on columns
cols_to_append_UCDP = ['count_num_daily_events', 'deaths_civilians', 'deaths_unknown', 'best_est', 'high_est',
                  'low_est', 'deaths_a', 'deaths_b'] # list(df_UCDP.columns)
cols_to_append_Phoe = ['GoldsteinScore', 'count_num_daily_events_Phoe'] # list(df_Phoe.columns)
cols_to_append_GDELT = [] # to fill

cols_to_append_UCDP.extend(cols_to_append_Phoe)
cols_to_append_UCDP.extend(cols_to_append_GDELT)
cols_to_append = cols_to_append_UCDP

def append_empty_cols_to_df(df, col_names):
    for i in col_names:
        df[i] = np.nan
    return df
        
df = append_empty_cols_to_df(df, cols_to_append_UCDP)

In [247]:
# very slow, still performs ok for limited time frame?

def combine_empty_and_filled_df(df, df_agg, CC3, CCS, cols_to_append, CC_name1, CC_name2):
    for i in range(0, len(df)):
    #for i in range(0, 2000):
        if CC_name2 == 'CCS' and df.CC3[i] != 'PSE':
            df_temp = df_agg[(df_agg.date_start == df.date_start[i]) & (df_agg[CC_name1] == int(df[CC_name2][i]))]
        else:
            df_temp = df_agg[(df_agg.date_start == df.date_start[i]) & (df_agg[CC_name1] == df[CC_name2][i])]
                
                # further condition for GDELT?
                
        if df_temp.empty is False:
            for k in cols_to_append:
                df[k][i] = df_temp[k]
                df.is_copy = False
                #df.loc[i, k] = df_temp[k] #--> bug
    return df
                    

In [248]:
# combine empty DF & UCDP
cols_to_append_UCDP = ['count_num_daily_events', 'deaths_civilians', 'deaths_unknown', 'best_est', 'high_est',
                  'low_est', 'deaths_a', 'deaths_b']
country_filter_col_name = 'gwno'
country_filter_excel = 'CCS'
combined_df = combine_empty_and_filled_df(df, 
                                          df_UCDP, 
                                          CC3, 
                                          CCS, 
                                          cols_to_append_UCDP, 
                                          country_filter_col_name, 
                                          country_filter_excel)


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [249]:
# add Phoenix df to combined df
cols_to_append_Phoe = ['GoldsteinScore', 'count_num_daily_events_Phoe']
df_Phoe = df_Phoe.rename(columns={'Date': 'date_start', 'count_num_daily_events': 'count_num_daily_events_Phoe'})
country_filter_col_name = 'SourceActorFull'
country_filter_excel = 'CC3'
combined_df = combine_empty_and_filled_df(combined_df,
                                          df_Phoe,
                                          CC3, 
                                          CCS, 
                                          cols_to_append_Phoe, 
                                          country_filter_col_name, 
                                          country_filter_excel)

# add GDELT df to combined df
# cols_to_append_Phoe = ['', '']
#country_filter_col_name = ''
#country_filter_excel = 'CC2'
#combined_df = combine_empty_and_filled_df(combined_df, df_LT, CC3, CCS, cols_to_append_GDELT, country_filter_col_name, country_filter_excel)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [232]:
# little checks

result = df_Phoe[(df_Phoe.date_start == '2017-01-05') & (df_Phoe['SourceActorFull'] == 'SYR')]
list(result.itertuples(index=False))

print df_Phoe.GoldsteinScore.count()
print combined_df.GoldsteinScore.count()  

# subset df based on date
test = df_Phoe[(df_Phoe['date_start'] >= '2014-01-01') & (df_Phoe['SourceActorFull'] == 'PSE')]
test.GoldsteinScore.count()

[Pandas(date_start='2017-01-05', SourceActorFull='SYR', GoldsteinScore=58.0, count_num_daily_events_Phoe=16)]

In [251]:
# save combined df to csv
combined_df.to_csv(path +'combined_df.csv')